In [ ]:
import equinox as eqx
import jax
import jax.numpy as jnp
from rich.pretty import pprint

from squint.circuit import Circuit
from squint.ops.fock import (
    BeamSplitter,
    FockState,
    TwoModeWeakThermalSource,
)
from squint.utils import print_nonzero_entries

In [ ]:
n_phases = 1
wires_star = tuple(i for i in range(n_phases + 1))
wires_lab = tuple(i for i in range(n_phases + 1, 2 * (n_phases + 1)))

circuit = Circuit(backend="mixed")
circuit.add(
    TwoModeWeakCoherentSource(wires=(0, 1), epsilon=0.3, g=1.0, phi=0.2), "star"
)
# circuit.add(
#     FockState(
#         wires=wires_star,
#         n=[(1.0, tuple(1 if i == j else 0 for i in wires_star)) for j in wires_star]
#     )
# )
circuit.add(
    FockState(
        wires=wires_lab,
        n=[(1.0, tuple(1 if i == j else 0 for i in wires_lab)) for j in wires_lab],
    )
)
# for i in range(1, n_phases+1):
#     circuit.add(Phase(wires=(i,), phi=0.1), f"phase{i}")

for wire_star, wire_lab in zip(wires_star, wires_lab, strict=False):
    circuit.add(BeamSplitter(wires=(wire_star, wire_lab)))

# circuit.add(ErasureChannel(wires=(0,)))
pprint(circuit)
print(circuit.wires)